In [86]:
#!pip install catboost

In [1]:
import catboost as cb
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

from catboost import CatBoostRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
print(f"catBoost version: {cb.__version__}")
print(f"numpy version: {np.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"seaborn version: {sns.__version__}")

catBoost version: 1.2.2
numpy version: 1.26.0
pandas version: 2.1.1
seaborn version: 0.12.2


In [3]:
print('Loading Properties ...')
properties2016 = pd.read_csv('properties_2016.csv', low_memory = False)
properties2017 = pd.read_csv('properties_2017.csv', low_memory = False)

print('Loading Train ...')
train2016 = pd.read_csv('train_2016_v2.csv', parse_dates=['transactiondate'], low_memory=False)
train2017 = pd.read_csv('train_2017.csv', parse_dates=['transactiondate'], low_memory=False)

print('Loading Sample ...')
sample_submission = pd.read_csv('sample_submission.csv', low_memory = False)

print('Done ...')

Loading Properties ...
Loading Train ...
Loading Sample ...
Done ...


In [4]:
print(properties2016.shape)
print(properties2017.shape)
print(train2016.shape)
print(train2017.shape)
print(sample_submission.shape)

(2985217, 58)
(2985217, 58)
(90275, 3)
(77613, 3)
(2985217, 7)


## Exploring the csvs

In [5]:
train2016.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,1/1/2016
1,14366692,-0.1684,1/1/2016
2,12098116,-0.0040,1/1/2016
3,12643413,0.0218,2/1/2016
4,14432541,-0.0050,2/1/2016


In [6]:
train2017.head()

,parcelid,logerror,transactiondate
0,14297519,0.025595,1/1/2017
1,17052889,0.055619,1/1/2017
2,14186244,0.005383,1/1/2017
3,12177905,-0.103410,1/1/2017
4,10887214,0.006940,1/1/2017


In [7]:
properties2016.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [8]:
properties2017.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


In [9]:
sample_submission.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


## Observing sample submission parcelID order

In [10]:
sample_submission["ParcelId"].equals(properties2016["parcelid"])

True

In [11]:
sample_submission["ParcelId"].equals(properties2017["parcelid"])

False

The parcelid order in sample submission seems to follow that of 2016, thus we need to reorder 2017

### Reordering properties2017

In [12]:
properties2017 = pd.merge(sample_submission["ParcelId"], properties2017, how='left', left_on ='ParcelId',right_on='parcelid').drop("ParcelId", axis=1)
properties2017

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985212,168176230,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,110245.0,226900.0,2016.0,116655.0,3035.26,NaN,NaN,NaN
2985213,14273630,NaN,NaN,NaN,3.0,3.0,NaN,NaN,3.0,NaN,...,NaN,NaN,378466.0,762157.0,2016.0,383691.0,8653.10,NaN,NaN,NaN
2985214,168040630,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,NaN,NaN,201575.0,483188.0,2016.0,281613.0,5671.40,NaN,NaN,NaN
2985215,168040830,NaN,NaN,NaN,3.0,4.0,NaN,NaN,3.0,NaN,...,NaN,NaN,347863.0,578621.0,2016.0,230758.0,5894.36,NaN,NaN,NaN


In [13]:
sample_submission["ParcelId"].equals(properties2017["parcelid"])

True

## Splitting transaction dates into month, year (removing day)

In [14]:
def split_date(df):
    df["transactiondate"] = pd.to_datetime(df["transactiondate"], dayfirst=True)
    df["transaction_year"] = df["transactiondate"].dt.year
    
#     df["transaction_month"] = df["transactiondate"].dt.month

#     the code below is used if we want to let 2017 jan start as month 13 (might be useful when we combine both years to train a model)
    df["transaction_month"] = (df["transactiondate"].dt.year - 2016)*12 + df["transactiondate"].dt.month

    
#     df["transaction_day"] = df["transactiondate"].dt.day
    
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df

train2016 = split_date(train2016)
train2017 = split_date(train2017)

In [15]:
print(train2016.shape)
train2016.head()

(90275, 4)


,parcelid,logerror,transaction_year,transaction_month
0,11016594,0.0276,2016,1
1,14366692,-0.1684,2016,1
2,12098116,-0.0040,2016,1
3,12643413,0.0218,2016,1
4,14432541,-0.0050,2016,1


In [16]:
print(train2017.shape)
train2017.head()

(77613, 4)


,parcelid,logerror,transaction_year,transaction_month
0,14297519,0.025595,2017,13
1,17052889,0.055619,2017,13
2,14186244,0.005383,2017,13
3,12177905,-0.103410,2017,13
4,10887214,0.006940,2017,13


we can see that Jan 2017 starts as month 13

## Non-Null counts for the columns in properties.csv

In [17]:
properties2016.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 58 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   parcelid                      2985217 non-null  int64  
 1   airconditioningtypeid         811519 non-null   float64
 2   architecturalstyletypeid      6061 non-null     float64
 3   basementsqft                  1628 non-null     float64
 4   bathroomcnt                   2973755 non-null  float64
 5   bedroomcnt                    2973767 non-null  float64
 6   buildingclasstypeid           12629 non-null    float64
 7   buildingqualitytypeid         1938488 non-null  float64
 8   calculatedbathnbr             2856305 non-null  float64
 9   decktypeid                    17096 non-null    float64
 10  finishedfloor1squarefeet      202717 non-null   float64
 11  calculatedfinishedsquarefeet  2929652 non-null  float64
 12  finishedsquarefeet12        

## Possible cleaning of dataframe  

 ### 1) remove the features with 1 unique value 

In [18]:
oneUniqueValueList2016 = []

for i in properties2016.columns:
    uniqueNum = properties2016[i].nunique()
    
    if uniqueNum == 1:
        oneUniqueValueList2016.append(i)
    
    
oneUniqueValueList2016

['decktypeid',
 'hashottuborspa',
 'poolcnt',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'storytypeid',
 'fireplaceflag',
 'taxdelinquencyflag']

In [19]:
oneUniqueValueList2017 = []

for i in properties2017.columns:
    uniqueNum = properties2017[i].nunique()
    
    if uniqueNum == 1:
        oneUniqueValueList2017.append(i)
    
    
oneUniqueValueList2017

['decktypeid',
 'hashottuborspa',
 'poolcnt',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'storytypeid',
 'fireplaceflag',
 'taxdelinquencyflag']

Both 2016 and 2017 have the same columns that only has 1 unique value, thus we will be removing them

### Removing above elements from our df

In [20]:
properties2016.drop(oneUniqueValueList2016, inplace = True, axis=1)
properties2017.drop(oneUniqueValueList2017, inplace = True, axis=1)

print(properties2016.shape)
print(properties2017.shape)

(2985217, 49)
(2985217, 49)


 ### 2) Removing features with >X% Null values

#### 2016 Missing Percentages

In [21]:
percent_missing_2016 = properties2016.isnull().sum() * 100 / len(properties2016)
percent_missing_2016

parcelid                         0.000000
airconditioningtypeid           72.815410
architecturalstyletypeid        99.796966
basementsqft                    99.945465
bathroomcnt                      0.383959
bedroomcnt                       0.383557
buildingclasstypeid             99.576949
buildingqualitytypeid           35.063749
calculatedbathnbr                4.318346
finishedfloor1squarefeet        93.209304
calculatedfinishedsquarefeet     1.861339
finishedsquarefeet12             9.246664
finishedsquarefeet13            99.743000
finishedsquarefeet15            93.608572
finishedsquarefeet50            93.209304
finishedsquarefeet6             99.263002
fips                             0.383121
fireplacecnt                    89.527160
fullbathcnt                      4.318346
garagecarcnt                    70.411967
garagetotalsqft                 70.411967
heatingorsystemtypeid           39.488453
latitude                         0.383121
longitude                        0

Assuming X = 80

In [23]:
percent_missing_2016[percent_missing_2016 > 80]

architecturalstyletypeid    99.796966
basementsqft                99.945465
buildingclasstypeid         99.576949
finishedfloor1squarefeet    93.209304
finishedsquarefeet13        99.743000
finishedsquarefeet15        93.608572
finishedsquarefeet50        93.209304
finishedsquarefeet6         99.263002
fireplacecnt                89.527160
poolsizesum                 99.063385
threequarterbathnbr         89.560859
typeconstructiontypeid      99.773986
yardbuildingsqft17          97.308236
yardbuildingsqft26          99.911330
taxdelinquencyyear          98.108546
dtype: float64

#### 2017 Missing Percentages with X = 80

In [24]:
percent_missing_2017 = properties2017.isnull().sum() * 100 / len(properties2017)
percent_missing_2017[percent_missing_2017 > 80]

architecturalstyletypeid    99.796966
basementsqft                99.945498
buildingclasstypeid         99.573532
finishedfloor1squarefeet    93.174433
finishedsquarefeet13        99.743034
finishedsquarefeet15        93.629106
finishedsquarefeet50        93.174433
finishedsquarefeet6         99.280387
fireplacecnt                89.510846
poolsizesum                 99.063452
threequarterbathnbr         89.402546
typeconstructiontypeid      99.774020
yardbuildingsqft17          97.266932
yardbuildingsqft26          99.911363
taxdelinquencyyear          98.106771
dtype: float64

Both 2016 and 2017 properties also have the same columns that have more than 80% of its values missing. We will be removing them.

In [26]:
percentMissingList = list(percent_missing_2017[percent_missing_2017 > 80].index)
percentMissingList

['architecturalstyletypeid',
 'basementsqft',
 'buildingclasstypeid',
 'finishedfloor1squarefeet',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fireplacecnt',
 'poolsizesum',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'taxdelinquencyyear']

In [27]:
properties2016.drop(percentMissingList, inplace = True, axis=1)
properties2017.drop(percentMissingList, inplace = True, axis=1)

print(properties2016.shape)
print(properties2017.shape)

(2985217, 34)
(2985217, 34)


## Labelling our numerical & categorical columns

In [28]:
nominal_features = ["censustractandblock",
                    "airconditioningtypeid",
                    "fips",
                    "heatingorsystemtypeid",
                    "propertycountylandusecode",
                    "propertylandusetypeid",
                    "regionidcounty",
                    "rawcensustractandblock",
                    "regionidneighborhood",
                    "propertyzoningdesc",
                    "regionidcity",
                    "regionidzip",                  
                   ]

ordinal_features = ["bathroomcnt",
                    "bedroomcnt",
                    "buildingqualitytypeid",
                    "calculatedbathnbr",
                    "fullbathcnt",
                    "garagecarcnt",
                    "numberofstories",
                    "roomcnt",
                    "unitcnt",
                    "yearbuilt",
                    "assessmentyear"] 

categorical_features = nominal_features + ordinal_features

#we have to include the transaction date individual features that we created
categorical_features += ["transaction_year", "transaction_month"]

numeric_features = ["calculatedfinishedsquarefeet",
                    "finishedsquarefeet12",
                    "garagetotalsqft",
                    "latitude",
                    "longitude",
                    "lotsizesquarefeet",
                    "taxvaluedollarcnt",
                    "structuretaxvaluedollarcnt",
                    "landtaxvaluedollarcnt",
                    "taxamount"]

In [30]:
print(f"numeric features length: {len(numeric_features)}")
print(f"categorical features length: {len(categorical_features)}")

numeric features length: 10
categorical features length: 25


## Preparing our train and test data

- our first approach is to combine both 2016 and 2017 data and train a model that predicts both years together
- we have to merge train.csv with their respective properties.csv


In [31]:
# how = left is equivalent to a left join in SQL, thus we will only have a dataframe with houses that are sold
combined2016 = pd.merge(train2016, properties2016, how = 'left', on = 'parcelid')  
combined2017 = pd.merge(train2017, properties2017, how = 'left', on = 'parcelid')

print(combined2016.shape)
print(combined2017.shape)

(90275, 37)
(77613, 37)


In [32]:
#we can see the merging of both train and properties
combined2016.head()

,parcelid,logerror,transaction_year,transaction_month,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,...,roomcnt,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
0,11016594,0.0276,2016,1,1.0,2.0,3.0,4.0,2.0,1684.0,...,0.0,1.0,1959.0,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,6.037107e+13
1,14366692,-0.1684,2016,1,NaN,3.5,4.0,NaN,3.5,2263.0,...,0.0,NaN,2014.0,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN
2,12098116,-0.0040,2016,1,1.0,3.0,2.0,4.0,3.0,2217.0,...,0.0,1.0,1940.0,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,6.037464e+13
3,12643413,0.0218,2016,1,1.0,2.0,2.0,4.0,2.0,839.0,...,0.0,1.0,1987.0,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,6.037296e+13
4,14432541,-0.0050,2016,1,NaN,2.5,4.0,NaN,2.5,2283.0,...,8.0,NaN,1981.0,2.0,169574.0,434551.0,2015.0,264977.0,5488.96,6.059042e+13


In [33]:
# combining 2016 and 2017 for the final df
combined = pd.concat([combined2016, combined2017])
combined

,parcelid,logerror,transaction_year,transaction_month,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,...,roomcnt,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
0,11016594,0.027600,2016,1,1.0,2.0,3.0,4.0,2.0,1684.0,...,0.0,1.0,1959.0,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,6.037107e+13
1,14366692,-0.168400,2016,1,NaN,3.5,4.0,NaN,3.5,2263.0,...,0.0,NaN,2014.0,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN
2,12098116,-0.004000,2016,1,1.0,3.0,2.0,4.0,3.0,2217.0,...,0.0,1.0,1940.0,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,6.037464e+13
3,12643413,0.021800,2016,1,1.0,2.0,2.0,4.0,2.0,839.0,...,0.0,1.0,1987.0,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,6.037296e+13
4,14432541,-0.005000,2016,1,NaN,2.5,4.0,NaN,2.5,2283.0,...,8.0,NaN,1981.0,2.0,169574.0,434551.0,2015.0,264977.0,5488.96,6.059042e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77608,10833991,-0.002245,2017,21,1.0,3.0,3.0,8.0,3.0,1741.0,...,0.0,1.0,1980.0,NaN,265000.0,379000.0,2016.0,114000.0,4685.34,6.037132e+13
77609,11000655,0.020615,2017,21,NaN,2.0,2.0,6.0,2.0,1286.0,...,0.0,1.0,1940.0,NaN,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13
77610,17239384,0.013209,2017,21,NaN,2.0,4.0,NaN,2.0,1612.0,...,7.0,NaN,1964.0,1.0,50683.0,67205.0,2016.0,16522.0,1107.48,6.111008e+13
77611,12773139,0.037129,2017,21,1.0,1.0,3.0,4.0,1.0,1032.0,...,0.0,1.0,1954.0,NaN,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13


## Populating NaN values (0 for numerical columns, empty string for categorical columns)

In [34]:
combined[numeric_features] = combined[numeric_features].fillna(0)

In [35]:
combined[categorical_features] = combined[categorical_features].fillna('')

## Converting all categorical feature values to strings (a requirement for catBoost)

In [36]:
for i in categorical_features:
    combined[i] = combined[i].astype(str)

In [37]:
combined

,parcelid,logerror,transaction_year,transaction_month,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,...,roomcnt,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
0,11016594,0.027600,2016,1,1.0,2.0,3.0,4.0,2.0,1684.0,...,0.0,1.0,1959.0,,122754.0,360170.0,2015.0,237416.0,6735.88,60371066461001.0
1,14366692,-0.168400,2016,1,,3.5,4.0,,3.5,2263.0,...,0.0,,2014.0,,346458.0,585529.0,2015.0,239071.0,10153.02,
2,12098116,-0.004000,2016,1,1.0,3.0,2.0,4.0,3.0,2217.0,...,0.0,1.0,1940.0,,61994.0,119906.0,2015.0,57912.0,11484.48,60374638003004.0
3,12643413,0.021800,2016,1,1.0,2.0,2.0,4.0,2.0,839.0,...,0.0,1.0,1987.0,,171518.0,244880.0,2015.0,73362.0,3048.74,60372963002002.0
4,14432541,-0.005000,2016,1,,2.5,4.0,,2.5,2283.0,...,8.0,,1981.0,2.0,169574.0,434551.0,2015.0,264977.0,5488.96,60590423381006.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77608,10833991,-0.002245,2017,21,1.0,3.0,3.0,8.0,3.0,1741.0,...,0.0,1.0,1980.0,,265000.0,379000.0,2016.0,114000.0,4685.34,60371319001006.0
77609,11000655,0.020615,2017,21,,2.0,2.0,6.0,2.0,1286.0,...,0.0,1.0,1940.0,,70917.0,354621.0,2016.0,283704.0,4478.43,60371014003002.0
77610,17239384,0.013209,2017,21,,2.0,4.0,,2.0,1612.0,...,7.0,,1964.0,1.0,50683.0,67205.0,2016.0,16522.0,1107.48,61110084022016.0
77611,12773139,0.037129,2017,21,1.0,1.0,3.0,4.0,1.0,1032.0,...,0.0,1.0,1954.0,,32797.0,49546.0,2016.0,16749.0,876.43,60374338022005.0


## Getting our train / test splits


In [38]:
y = combined["logerror"]
X = combined.drop(["parcelid", "logerror"], axis=1)

In [39]:
print(X.shape)
X.head()

(167888, 35)


,transaction_year,transaction_month,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,roomcnt,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
0,2016,1,1.0,2.0,3.0,4.0,2.0,1684.0,1684.0,6037.0,...,0.0,1.0,1959.0,,122754.0,360170.0,2015.0,237416.0,6735.88,60371066461001.0
1,2016,1,,3.5,4.0,,3.5,2263.0,2263.0,6059.0,...,0.0,,2014.0,,346458.0,585529.0,2015.0,239071.0,10153.02,
2,2016,1,1.0,3.0,2.0,4.0,3.0,2217.0,2217.0,6037.0,...,0.0,1.0,1940.0,,61994.0,119906.0,2015.0,57912.0,11484.48,60374638003004.0
3,2016,1,1.0,2.0,2.0,4.0,2.0,839.0,839.0,6037.0,...,0.0,1.0,1987.0,,171518.0,244880.0,2015.0,73362.0,3048.74,60372963002002.0
4,2016,1,,2.5,4.0,,2.5,2283.0,2283.0,6059.0,...,8.0,,1981.0,2.0,169574.0,434551.0,2015.0,264977.0,5488.96,60590423381006.0


In [40]:
y.head()

0    0.0276
1   -0.1684
2   -0.0040
3    0.0218
4   -0.0050
Name: logerror, dtype: float64

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

print(f"X_train size: {len(X_train)}")
print(f"X_test size: {len(X_test)}")
print(f"y_train size: {len(y_train)}")
print(f"y_test size: {len(y_test)}")

X_train size: 134310
X_test size: 33578
y_train size: 134310
y_test size: 33578


In [42]:
X_train.head()

,transaction_year,transaction_month,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,roomcnt,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
1145,2016,1,,3.0,2.0,1.0,3.0,1831.0,1831.0,6037.0,...,0.0,1.0,1981.0,,254938.0,410564.0,2015.0,155626.0,5008.74,60372641031011.0
46858,2017,18,,2.0,3.0,6.0,2.0,1571.0,1571.0,6037.0,...,0.0,1.0,1947.0,,35960.0,108540.0,2016.0,72580.0,1353.25,60371411021004.0
26074,2017,16,,2.5,3.0,,2.5,2174.0,2174.0,6111.0,...,7.0,,1990.0,2.0,256156.0,512312.0,2016.0,256156.0,5692.00,61110076141139.0
43052,2016,6,,2.0,3.0,4.0,2.0,1809.0,1809.0,6037.0,...,0.0,1.0,1926.0,,97580.0,166558.0,2015.0,68978.0,2174.84,60372141004000.0
76357,2016,9,,1.0,1.0,,1.0,600.0,600.0,6059.0,...,3.0,,1930.0,2.0,10547.0,410000.0,2015.0,399453.0,4320.64,60590320111002.0


In [43]:
y_train.head()

1145     0.045000
46858   -2.271428
26074    0.008852
43052   -0.098700
76357    0.035400
Name: logerror, dtype: float64

## Before we train a catBoost model, we need to get indices of categorical columns from X_train

In [44]:
categorical_indices = []
for i in categorical_features:
    idx = X_train.columns.get_loc(i)
    categorical_indices.append(idx)

categorical_indices.sort()
       
print(categorical_indices)

[0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 13, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 34]


## Preparing a submission onto kaggle

In [45]:
datelist = list(sample_submission.columns[1:])
print(datelist)
sample_submission

['201610', '201611', '201612', '201710', '201711', '201712']


,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0
...,...,...,...,...,...,...,...
2985212,168176230,0,0,0,0,0,0
2985213,14273630,0,0,0,0,0,0
2985214,168040630,0,0,0,0,0,0
2985215,168040830,0,0,0,0,0,0


## Function to preprocess our data as we did above

In [46]:
def preprocess(df):
    df[numeric_features] = df[numeric_features].fillna(0)
    df[categorical_features] = df[categorical_features].fillna('')
    
    for i in categorical_features:
        df[i] = df[i].astype(str)

def makeTransactionDF(df, month, year):
    data = {'transaction_year': [year for i in range(len(df))],
            'transaction_month': [month for i in range(len(df))]}
    
    transaction_month_year = pd.DataFrame(data)
    test_month_year = pd.concat([transaction_month_year, df], axis=1)
    
    return test_month_year

---
# Using oct 2016 to predict for all 6 time points 

1) Making a transaction df for October 2016

In [47]:
test_oct_2016 = makeTransactionDF(properties2016, 10, 2016)

In [48]:
X_test_oct_2016 = test_oct_2016.drop('parcelid', axis=1)

print(X_test_oct_2016.shape)
X_test_oct_2016.head()

(2985217, 35)


,transaction_year,transaction_month,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,roomcnt,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
0,2016,10,NaN,0.0,0.0,NaN,NaN,NaN,NaN,6037.0,...,0.0,NaN,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN
1,2016,10,NaN,0.0,0.0,NaN,NaN,NaN,NaN,6037.0,...,0.0,NaN,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN
2,2016,10,NaN,0.0,0.0,NaN,NaN,73026.0,NaN,6037.0,...,0.0,2.0,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN
3,2016,10,NaN,0.0,0.0,7.0,NaN,5068.0,NaN,6037.0,...,0.0,NaN,1948.0,1.0,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN
4,2016,10,NaN,0.0,0.0,NaN,NaN,1776.0,NaN,6037.0,...,0.0,1.0,1947.0,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN


In [49]:
preprocess(X_test_oct_2016)

In [50]:
X_test_oct_2016

,transaction_year,transaction_month,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,roomcnt,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
0,2016,10,,0.0,0.0,,,0.0,0.0,6037.0,...,0.0,,,,0.0,9.0,2015.0,9.0,0.00,
1,2016,10,,0.0,0.0,,,0.0,0.0,6037.0,...,0.0,,,,0.0,27516.0,2015.0,27516.0,0.00,
2,2016,10,,0.0,0.0,,,73026.0,0.0,6037.0,...,0.0,2.0,,,650756.0,1413387.0,2015.0,762631.0,20800.37,
3,2016,10,,0.0,0.0,7.0,,5068.0,0.0,6037.0,...,0.0,,1948.0,1.0,571346.0,1156834.0,2015.0,585488.0,14557.57,
4,2016,10,,0.0,0.0,,,1776.0,0.0,6037.0,...,0.0,1.0,1947.0,,193796.0,433491.0,2015.0,239695.0,5725.17,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985212,2016,10,,,,,,0.0,0.0,,...,,,,,0.0,0.0,,0.0,0.00,
2985213,2016,10,,,,,,0.0,0.0,,...,,,,,0.0,0.0,,0.0,0.00,
2985214,2016,10,,,,,,0.0,0.0,,...,,,,,0.0,0.0,,0.0,0.00,
2985215,2016,10,,,,,,0.0,0.0,,...,,,,,0.0,0.0,,0.0,0.00,


# Training and Prediction

## With Ensemble (1000 iterations)

In [51]:
num_ensembles = 5
y_pred = 0.0
for i in tqdm(range(num_ensembles)):
    model = CatBoostRegressor(
        iterations=1000, 
        learning_rate=0.03,
        depth=6, 
        l2_leaf_reg=3,
        loss_function='MAE',
        eval_metric='MAE',
        cat_features = categorical_indices,
        random_seed=i)
    
    model.fit(X_train, y_train)

    y_pred += model.predict(X_test_oct_2016)
    
y_pred /= num_ensembles

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

0:	learn: 0.0688202	total: 493ms	remaining: 8m 12s
1:	learn: 0.0688051	total: 829ms	remaining: 6m 53s
2:	learn: 0.0687898	total: 1.21s	remaining: 6m 41s
3:	learn: 0.0687742	total: 1.58s	remaining: 6m 34s
4:	learn: 0.0687588	total: 1.94s	remaining: 6m 26s
5:	learn: 0.0687445	total: 2.34s	remaining: 6m 27s
6:	learn: 0.0687321	total: 2.66s	remaining: 6m 17s
7:	learn: 0.0687165	total: 3s	remaining: 6m 12s
8:	learn: 0.0687032	total: 3.4s	remaining: 6m 14s
9:	learn: 0.0686918	total: 3.73s	remaining: 6m 9s
10:	learn: 0.0686793	total: 4.06s	remaining: 6m 5s
11:	learn: 0.0686648	total: 4.43s	remaining: 6m 4s
12:	learn: 0.0686521	total: 4.79s	remaining: 6m 3s
13:	learn: 0.0686425	total: 5.1s	remaining: 5m 59s
14:	learn: 0.0686322	total: 5.41s	remaining: 5m 55s
15:	learn: 0.0686240	total: 5.77s	remaining: 5m 54s
16:	learn: 0.0686122	total: 6.13s	remaining: 5m 54s
17:	learn: 0.0686020	total: 6.49s	remaining: 5m 54s
18:	learn: 0.0685915	total: 6.83s	remaining: 5m 52s
19:	learn: 0.0685815	total: 7.1

158:	learn: 0.0680863	total: 55.7s	remaining: 4m 54s
159:	learn: 0.0680846	total: 56.1s	remaining: 4m 54s
160:	learn: 0.0680833	total: 56.4s	remaining: 4m 53s
161:	learn: 0.0680821	total: 56.8s	remaining: 4m 53s
162:	learn: 0.0680798	total: 57.1s	remaining: 4m 53s
163:	learn: 0.0680785	total: 57.4s	remaining: 4m 52s
164:	learn: 0.0680768	total: 57.7s	remaining: 4m 52s
165:	learn: 0.0680751	total: 58s	remaining: 4m 51s
166:	learn: 0.0680732	total: 58.4s	remaining: 4m 51s
167:	learn: 0.0680705	total: 58.7s	remaining: 4m 50s
168:	learn: 0.0680683	total: 59.1s	remaining: 4m 50s
169:	learn: 0.0680669	total: 59.4s	remaining: 4m 49s
170:	learn: 0.0680655	total: 59.8s	remaining: 4m 49s
171:	learn: 0.0680641	total: 1m	remaining: 4m 49s
172:	learn: 0.0680603	total: 1m	remaining: 4m 49s
173:	learn: 0.0680592	total: 1m	remaining: 4m 48s
174:	learn: 0.0680577	total: 1m 1s	remaining: 4m 48s
175:	learn: 0.0680563	total: 1m 1s	remaining: 4m 48s
176:	learn: 0.0680553	total: 1m 1s	remaining: 4m 47s
177:

312:	learn: 0.0678904	total: 1m 48s	remaining: 3m 58s
313:	learn: 0.0678885	total: 1m 48s	remaining: 3m 58s
314:	learn: 0.0678880	total: 1m 49s	remaining: 3m 57s
315:	learn: 0.0678870	total: 1m 49s	remaining: 3m 57s
316:	learn: 0.0678860	total: 1m 49s	remaining: 3m 56s
317:	learn: 0.0678852	total: 1m 50s	remaining: 3m 56s
318:	learn: 0.0678846	total: 1m 50s	remaining: 3m 56s
319:	learn: 0.0678840	total: 1m 50s	remaining: 3m 55s
320:	learn: 0.0678825	total: 1m 51s	remaining: 3m 55s
321:	learn: 0.0678815	total: 1m 51s	remaining: 3m 55s
322:	learn: 0.0678805	total: 1m 52s	remaining: 3m 54s
323:	learn: 0.0678793	total: 1m 52s	remaining: 3m 54s
324:	learn: 0.0678781	total: 1m 52s	remaining: 3m 54s
325:	learn: 0.0678775	total: 1m 53s	remaining: 3m 53s
326:	learn: 0.0678764	total: 1m 53s	remaining: 3m 53s
327:	learn: 0.0678760	total: 1m 53s	remaining: 3m 53s
328:	learn: 0.0678756	total: 1m 54s	remaining: 3m 52s
329:	learn: 0.0678754	total: 1m 54s	remaining: 3m 52s
330:	learn: 0.0678752	total:

465:	learn: 0.0677042	total: 2m 44s	remaining: 3m 7s
466:	learn: 0.0677018	total: 2m 44s	remaining: 3m 7s
467:	learn: 0.0677006	total: 2m 44s	remaining: 3m 7s
468:	learn: 0.0676996	total: 2m 45s	remaining: 3m 7s
469:	learn: 0.0676990	total: 2m 45s	remaining: 3m 6s
470:	learn: 0.0676981	total: 2m 45s	remaining: 3m 6s
471:	learn: 0.0676976	total: 2m 46s	remaining: 3m 5s
472:	learn: 0.0676973	total: 2m 46s	remaining: 3m 5s
473:	learn: 0.0676958	total: 2m 46s	remaining: 3m 5s
474:	learn: 0.0676942	total: 2m 47s	remaining: 3m 4s
475:	learn: 0.0676935	total: 2m 47s	remaining: 3m 4s
476:	learn: 0.0676922	total: 2m 48s	remaining: 3m 4s
477:	learn: 0.0676913	total: 2m 48s	remaining: 3m 3s
478:	learn: 0.0676897	total: 2m 48s	remaining: 3m 3s
479:	learn: 0.0676891	total: 2m 49s	remaining: 3m 3s
480:	learn: 0.0676881	total: 2m 49s	remaining: 3m 3s
481:	learn: 0.0676871	total: 2m 50s	remaining: 3m 2s
482:	learn: 0.0676858	total: 2m 50s	remaining: 3m 2s
483:	learn: 0.0676832	total: 2m 50s	remaining:

...

464:	learn: 0.0677238	total: 2m 54s	remaining: 3m 20s
465:	learn: 0.0677226	total: 2m 54s	remaining: 3m 20s
466:	learn: 0.0677223	total: 2m 55s	remaining: 3m 19s
467:	learn: 0.0677211	total: 2m 55s	remaining: 3m 19s
468:	learn: 0.0677198	total: 2m 55s	remaining: 3m 18s
469:	learn: 0.0677184	total: 2m 56s	remaining: 3m 18s
470:	learn: 0.0677167	total: 2m 56s	remaining: 3m 18s
471:	learn: 0.0677153	total: 2m 56s	remaining: 3m 17s
472:	learn: 0.0677129	total: 2m 57s	remaining: 3m 17s
473:	learn: 0.0677111	total: 2m 57s	remaining: 3m 16s
474:	learn: 0.0677087	total: 2m 57s	remaining: 3m 16s
475:	learn: 0.0677082	total: 2m 58s	remaining: 3m 16s
476:	learn: 0.0677071	total: 2m 58s	remaining: 3m 15s
477:	learn: 0.0677059	total: 2m 59s	remaining: 3m 15s
478:	learn: 0.0677046	total: 2m 59s	remaining: 3m 15s
479:	learn: 0.0677034	total: 2m 59s	remaining: 3m 14s
480:	learn: 0.0677020	total: 3m	remaining: 3m 14s
481:	learn: 0.0677014	total: 3m	remaining: 3m 14s
482:	learn: 0.0677004	total: 3m 1s	r

617:	learn: 0.0675587	total: 3m 53s	remaining: 2m 24s
618:	learn: 0.0675576	total: 3m 53s	remaining: 2m 23s
619:	learn: 0.0675556	total: 3m 53s	remaining: 2m 23s
620:	learn: 0.0675550	total: 3m 54s	remaining: 2m 22s
621:	learn: 0.0675545	total: 3m 54s	remaining: 2m 22s
622:	learn: 0.0675522	total: 3m 55s	remaining: 2m 22s
623:	learn: 0.0675507	total: 3m 55s	remaining: 2m 21s
624:	learn: 0.0675502	total: 3m 55s	remaining: 2m 21s
625:	learn: 0.0675492	total: 3m 56s	remaining: 2m 21s
626:	learn: 0.0675482	total: 3m 56s	remaining: 2m 20s
627:	learn: 0.0675476	total: 3m 56s	remaining: 2m 20s
628:	learn: 0.0675472	total: 3m 57s	remaining: 2m 19s
629:	learn: 0.0675468	total: 3m 57s	remaining: 2m 19s
630:	learn: 0.0675455	total: 3m 57s	remaining: 2m 19s
631:	learn: 0.0675450	total: 3m 58s	remaining: 2m 18s
632:	learn: 0.0675438	total: 3m 58s	remaining: 2m 18s
633:	learn: 0.0675430	total: 3m 59s	remaining: 2m 17s
634:	learn: 0.0675423	total: 3m 59s	remaining: 2m 17s
635:	learn: 0.0675417	total:

771:	learn: 0.0674194	total: 4m 50s	remaining: 1m 25s
772:	learn: 0.0674185	total: 4m 50s	remaining: 1m 25s
773:	learn: 0.0674177	total: 4m 51s	remaining: 1m 25s
774:	learn: 0.0674170	total: 4m 51s	remaining: 1m 24s
775:	learn: 0.0674162	total: 4m 51s	remaining: 1m 24s
776:	learn: 0.0674157	total: 4m 52s	remaining: 1m 23s
777:	learn: 0.0674153	total: 4m 52s	remaining: 1m 23s
778:	learn: 0.0674145	total: 4m 52s	remaining: 1m 23s
779:	learn: 0.0674138	total: 4m 53s	remaining: 1m 22s
780:	learn: 0.0674133	total: 4m 53s	remaining: 1m 22s
781:	learn: 0.0674129	total: 4m 54s	remaining: 1m 21s
782:	learn: 0.0674120	total: 4m 54s	remaining: 1m 21s
783:	learn: 0.0674101	total: 4m 54s	remaining: 1m 21s
784:	learn: 0.0674093	total: 4m 55s	remaining: 1m 20s
785:	learn: 0.0674080	total: 4m 55s	remaining: 1m 20s
786:	learn: 0.0674072	total: 4m 55s	remaining: 1m 20s
787:	learn: 0.0674069	total: 4m 56s	remaining: 1m 19s
788:	learn: 0.0674060	total: 4m 56s	remaining: 1m 19s
789:	learn: 0.0674053	total:

926:	learn: 0.0673021	total: 5m 47s	remaining: 27.4s
927:	learn: 0.0673016	total: 5m 48s	remaining: 27s
928:	learn: 0.0673008	total: 5m 48s	remaining: 26.6s
929:	learn: 0.0673003	total: 5m 48s	remaining: 26.3s
930:	learn: 0.0672999	total: 5m 49s	remaining: 25.9s
931:	learn: 0.0672993	total: 5m 49s	remaining: 25.5s
932:	learn: 0.0672989	total: 5m 50s	remaining: 25.2s
933:	learn: 0.0672982	total: 5m 50s	remaining: 24.8s
934:	learn: 0.0672969	total: 5m 51s	remaining: 24.4s
935:	learn: 0.0672961	total: 5m 51s	remaining: 24s
936:	learn: 0.0672958	total: 5m 51s	remaining: 23.6s
937:	learn: 0.0672950	total: 5m 52s	remaining: 23.3s
938:	learn: 0.0672945	total: 5m 52s	remaining: 22.9s
939:	learn: 0.0672924	total: 5m 52s	remaining: 22.5s
940:	learn: 0.0672916	total: 5m 53s	remaining: 22.1s
941:	learn: 0.0672911	total: 5m 53s	remaining: 21.8s
942:	learn: 0.0672905	total: 5m 54s	remaining: 21.4s
943:	learn: 0.0672892	total: 5m 54s	remaining: 21s
944:	learn: 0.0672880	total: 5m 54s	remaining: 20.7s

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [35:30<00:00, 426.12s/it]


In [52]:
for i in datelist:
    sample_submission[i] = y_pred

In [53]:
sample_submission.to_csv("submission_ensemble_1000iterations_80percentNANRemoved.csv", index=False)